# Credit Risk Model - Exploratory Data Analysis

This notebook performs initial exploratory data analysis on the credit risk datasets:
- Loading and examining application train data
- Loading and examining bureau data
- Basic data quality checks
- Missing value analysis

## Data Loading and Initial Examination

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up plotting style
plt.style.use('seaborn')
sns.set_palette("husl")

# Display all columns and format floats
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set up data paths
DATA_DIR = Path('../data/raw')
APPLICATION_TRAIN_PATH = DATA_DIR / 'application_train.csv'
BUREAU_PATH = DATA_DIR / 'bureau.csv'

## Application Train Data

Let's load and examine the application train dataset first.

In [ ]:
# Load application train data
application_train = pd.read_csv(APPLICATION_TRAIN_PATH)

# Display first 5 rows
print("First 5 rows of application_train:")
display(application_train.head())

# Display basic information about the dataset
print("\nDataset Info:")
display(application_train.info())

# Display statistical summary
print("\nStatistical Summary:")
display(application_train.describe())

## Missing Value Analysis

Let's analyze the missing values in the application train dataset.

In [ ]:
# Calculate missing values
missing_values = application_train.isnull().sum()
missing_percentage = (missing_values / len(application_train)) * 100

# Create a summary DataFrame of missing values
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing Percentage': missing_percentage
})

# Sort by missing percentage in descending order and display only columns with missing values
missing_summary = missing_summary[missing_summary['Missing Values'] > 0].sort_values(
    'Missing Percentage', 
    ascending=False
)

print("Columns with missing values:")
display(missing_summary)

## Bureau Data

Now let's examine the bureau data.

In [ ]:
# Load bureau data
bureau = pd.read_csv(BUREAU_PATH)

# Display first 5 rows
print("First 5 rows of bureau data:")
display(bureau.head())

# Display basic information about the dataset
print("\nDataset Info:")
display(bureau.info())

# Display statistical summary
print("\nStatistical Summary:")
display(bureau.describe())

# E-commerce Features Prototype

This section demonstrates how to engineer e-commerce features that could be integrated into the credit risk model to enhance prediction accuracy using alternative data sources.

## RFM Analysis and Feature Engineering

We'll use the Online Retail dataset to create features like:
- **Recency**: Days since last purchase
- **Frequency**: Number of transactions
- **Monetary**: Total spending amount
- **Average Order Value**: Mean transaction value
- **Purchase Frequency**: Transactions per time period

In [ ]:
# Import additional libraries for e-commerce analysis
import requests
import zipfile
import io
from datetime import datetime, timedelta

# Download the Online Retail dataset from UCI ML Repository
def download_online_retail_data():
    """Download and extract the Online Retail dataset"""
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
    
    print("Downloading Online Retail dataset...")
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        # Save to data/raw directory
        retail_path = "../data/raw/online_retail.xlsx"
        with open(retail_path, 'wb') as f:
            f.write(response.content)
        
        print(f"Dataset downloaded successfully to {retail_path}")
        return retail_path
    except Exception as e:
        print(f"Error downloading dataset: {e}")
        # Create sample data if download fails
        return create_sample_retail_data()

def create_sample_retail_data():
    """Create sample e-commerce data for demonstration"""
    print("Creating sample e-commerce data...")
    
    np.random.seed(42)
    n_customers = 1000
    n_transactions = 5000
    
    # Generate sample data
    customer_ids = np.random.randint(1, n_customers + 1, n_transactions)
    invoice_dates = pd.date_range(start='2022-01-01', end='2023-12-31', periods=n_transactions)
    quantities = np.random.randint(1, 50, n_transactions)
    unit_prices = np.random.uniform(1, 100, n_transactions)
    
    sample_data = pd.DataFrame({
        'CustomerID': customer_ids,
        'InvoiceDate': invoice_dates,
        'Quantity': quantities,
        'UnitPrice': unit_prices,
        'TotalAmount': quantities * unit_prices
    })
    
    # Remove some customers to simulate missing CustomerID
    sample_data.loc[sample_data.index % 10 == 0, 'CustomerID'] = np.nan
    
    return sample_data

# Download or create the dataset
try:
    retail_file = download_online_retail_data()
    if retail_file.endswith('.xlsx'):
        retail_data = pd.read_excel(retail_file)
    else:
        retail_data = retail_file  # Sample data
except:
    retail_data = create_sample_retail_data()

print("Dataset loaded successfully!")

In [ ]:
# Data exploration and cleaning
print("Dataset shape:", retail_data.shape)
print("\nColumn names:", retail_data.columns.tolist())
print("\nFirst few rows:")
display(retail_data.head())

print("\nDataset info:")
retail_data.info()

print("\nMissing values:")
print(retail_data.isnull().sum())

# Clean the data
if 'InvoiceNo' in retail_data.columns:
    # For real Online Retail dataset
    retail_clean = retail_data.copy()
    
    # Calculate TotalAmount if not present
    if 'TotalAmount' not in retail_clean.columns:
        retail_clean['TotalAmount'] = retail_clean['Quantity'] * retail_clean['UnitPrice']
    
    # Remove rows with missing CustomerID
    retail_clean = retail_clean.dropna(subset=['CustomerID'])
    
    # Remove cancelled orders (negative quantities)
    retail_clean = retail_clean[retail_clean['Quantity'] > 0]
    retail_clean = retail_clean[retail_clean['UnitPrice'] > 0]
    
    # Convert CustomerID to integer
    retail_clean['CustomerID'] = retail_clean['CustomerID'].astype(int)
    
else:
    # For sample data
    retail_clean = retail_data.dropna(subset=['CustomerID'])
    retail_clean['CustomerID'] = retail_clean['CustomerID'].astype(int)

print(f"\nCleaned dataset shape: {retail_clean.shape}")
print(f"Number of unique customers: {retail_clean['CustomerID'].nunique()}")
print(f"Date range: {retail_clean['InvoiceDate'].min()} to {retail_clean['InvoiceDate'].max()}")

In [ ]:
# Calculate RFM (Recency, Frequency, Monetary) features
def calculate_rfm_features(data, customer_col='CustomerID', date_col='InvoiceDate', amount_col='TotalAmount'):
    """
    Calculate RFM features for each customer
    
    Args:
        data: DataFrame with transaction data
        customer_col: Column name for customer identifier
        date_col: Column name for transaction date
        amount_col: Column name for transaction amount
    
    Returns:
        DataFrame with RFM features per customer
    """
    
    # Get the reference date (latest date + 1 day)
    reference_date = data[date_col].max() + timedelta(days=1)
    
    # Group by customer and calculate RFM metrics
    rfm = data.groupby(customer_col).agg({
        date_col: lambda x: (reference_date - x.max()).days,  # Recency
        amount_col: ['count', 'sum', 'mean']  # Frequency, Monetary, AOV
    }).round(2)
    
    # Flatten column names
    rfm.columns = ['Recency', 'Frequency', 'Monetary', 'AvgOrderValue']
    
    # Reset index to make CustomerID a column
    rfm = rfm.reset_index()
    
    return rfm

# Calculate RFM features
rfm_features = calculate_rfm_features(retail_clean)

print("RFM Features calculated successfully!")
print(f"Shape: {rfm_features.shape}")
print("\nRFM Features Summary:")
display(rfm_features.describe())

print("\nFirst 10 customers:")
display(rfm_features.head(10))

In [ ]:
# Engineer additional e-commerce features
def engineer_ecommerce_features(data, rfm_data, customer_col='CustomerID', date_col='InvoiceDate', amount_col='TotalAmount'):
    """
    Engineer additional e-commerce features beyond basic RFM
    """
    
    # Calculate additional features per customer
    additional_features = data.groupby(customer_col).agg({
        date_col: [
            lambda x: x.nunique(),  # Unique shopping days
            lambda x: (x.max() - x.min()).days if len(x) > 1 else 0,  # Customer lifespan
        ],
        amount_col: [
            'std',  # Spending volatility
            lambda x: x.quantile(0.75) - x.quantile(0.25),  # IQR of spending
        ]
    }).round(2)
    
    # Flatten column names
    additional_features.columns = ['UniqueDays', 'CustomerLifespan', 'SpendingVolatility', 'SpendingIQR']
    additional_features = additional_features.reset_index()
    
    # Merge with RFM data
    enhanced_features = rfm_data.merge(additional_features, on=customer_col, how='left')
    
    # Calculate derived features
    enhanced_features['PurchaseFrequency'] = enhanced_features['Frequency'] / (enhanced_features['CustomerLifespan'] + 1)
    enhanced_features['DaysBetweenPurchases'] = enhanced_features['CustomerLifespan'] / (enhanced_features['Frequency'] - 1)
    enhanced_features['DaysBetweenPurchases'] = enhanced_features['DaysBetweenPurchases'].fillna(0)
    
    # Create RFM scores (1-5 scale)
    for col in ['Recency', 'Frequency', 'Monetary']:
        if col == 'Recency':
            # For recency, lower is better
            enhanced_features[f'{col}_Score'] = pd.qcut(enhanced_features[col], 5, labels=[5,4,3,2,1])
        else:
            # For frequency and monetary, higher is better
            enhanced_features[f'{col}_Score'] = pd.qcut(enhanced_features[col].rank(method='first'), 5, labels=[1,2,3,4,5])
    
    # Create combined RFM score
    enhanced_features['RFM_Score'] = (
        enhanced_features['Recency_Score'].astype(int) * 100 +
        enhanced_features['Frequency_Score'].astype(int) * 10 +
        enhanced_features['Monetary_Score'].astype(int)
    )
    
    # Customer segmentation based on RFM scores
    def segment_customers(row):
        if row['RFM_Score'] >= 444:
            return 'Champions'
        elif row['RFM_Score'] >= 334:
            return 'Loyal Customers'
        elif row['RFM_Score'] >= 224:
            return 'Potential Loyalists'
        elif row['RFM_Score'] >= 144:
            return 'At Risk'
        else:
            return 'Lost'
    
    enhanced_features['CustomerSegment'] = enhanced_features.apply(segment_customers, axis=1)
    
    return enhanced_features

# Generate enhanced e-commerce features
ecommerce_features = engineer_ecommerce_features(retail_clean, rfm_features)

print("Enhanced E-commerce Features:")
print(f"Shape: {ecommerce_features.shape}")
print(f"Features: {list(ecommerce_features.columns)}")

print("\nCustomer Segmentation:")
print(ecommerce_features['CustomerSegment'].value_counts())

print("\nFeature Summary:")
display(ecommerce_features.describe())

In [ ]:
# Visualize the distribution of e-commerce features
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('E-commerce Features Distribution', fontsize=16, fontweight='bold')

# List of numerical features to plot
numerical_features = [
    'Recency', 'Frequency', 'Monetary', 'AvgOrderValue',
    'PurchaseFrequency', 'SpendingVolatility', 'CustomerLifespan',
    'UniqueDays', 'DaysBetweenPurchases'
]

# Create distribution plots
for i, feature in enumerate(numerical_features):
    row = i // 3
    col = i % 3
    
    # Histogram
    axes[row, col].hist(ecommerce_features[feature].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[row, col].set_title(f'{feature} Distribution')
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel('Frequency')
    
    # Add statistics text
    mean_val = ecommerce_features[feature].mean()
    median_val = ecommerce_features[feature].median()
    axes[row, col].axvline(mean_val, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_val:.2f}')
    axes[row, col].axvline(median_val, color='green', linestyle='--', alpha=0.7, label=f'Median: {median_val:.2f}')
    axes[row, col].legend()

plt.tight_layout()
plt.show()

# RFM Score distribution
plt.figure(figsize=(12, 8))

# Customer segments pie chart
plt.subplot(2, 2, 1)
segment_counts = ecommerce_features['CustomerSegment'].value_counts()
plt.pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Customer Segmentation Distribution')

# RFM Score histogram
plt.subplot(2, 2, 2)
plt.hist(ecommerce_features['RFM_Score'], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
plt.title('RFM Score Distribution')
plt.xlabel('RFM Score')
plt.ylabel('Number of Customers')

# Recency vs Monetary scatter plot
plt.subplot(2, 2, 3)
scatter = plt.scatter(ecommerce_features['Recency'], ecommerce_features['Monetary'], 
                     c=ecommerce_features['Frequency'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Frequency')
plt.xlabel('Recency (Days)')
plt.ylabel('Monetary Value')
plt.title('Recency vs Monetary (colored by Frequency)')

# Average Order Value vs Purchase Frequency
plt.subplot(2, 2, 4)
plt.scatter(ecommerce_features['AvgOrderValue'], ecommerce_features['PurchaseFrequency'], alpha=0.6)
plt.xlabel('Average Order Value')
plt.ylabel('Purchase Frequency')
plt.title('AOV vs Purchase Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis and feature importance
# Select numerical features for correlation analysis
numerical_cols = [
    'Recency', 'Frequency', 'Monetary', 'AvgOrderValue',
    'PurchaseFrequency', 'SpendingVolatility', 'CustomerLifespan',
    'UniqueDays', 'DaysBetweenPurchases'
]

# Calculate correlation matrix
correlation_matrix = ecommerce_features[numerical_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('E-commerce Features Correlation Matrix')
plt.tight_layout()
plt.show()

# Feature statistics summary table
feature_summary = pd.DataFrame({
    'Feature': numerical_cols,
    'Mean': [ecommerce_features[col].mean() for col in numerical_cols],
    'Std': [ecommerce_features[col].std() for col in numerical_cols],
    'Min': [ecommerce_features[col].min() for col in numerical_cols],
    'Max': [ecommerce_features[col].max() for col in numerical_cols],
    'Skewness': [ecommerce_features[col].skew() for col in numerical_cols]
}).round(3)

print("E-commerce Features Summary Statistics:")
display(feature_summary)

# Credit risk implications
print("\n" + "="*60)
print("CREDIT RISK MODELING IMPLICATIONS")
print("="*60)

print("""
These e-commerce features can enhance credit risk models by providing insights into:

1. PAYMENT BEHAVIOR INDICATORS:
   • High Recency → Recent shopping activity suggests active financial engagement
   • High Frequency → Regular purchasing patterns indicate income stability
   • High Monetary → Higher spending capacity suggests better financial position

2. FINANCIAL STABILITY METRICS:
   • Low SpendingVolatility → Consistent spending patterns indicate financial discipline
   • Regular PurchaseFrequency → Predictable behavior reduces default risk
   • Long CustomerLifespan → Loyalty and long-term relationship building

3. RISK SEGMENTATION:
   • Champions → Lowest risk, highest creditworthiness
   • Loyal Customers → Low risk, stable payment behavior
   • At Risk/Lost → Higher risk, may indicate financial distress

4. PREDICTIVE FEATURES FOR INTEGRATION:
   • AvgOrderValue can predict disposable income levels
   • PurchaseFrequency indicates financial planning capability
   • CustomerSegment provides risk-based customer classification

NEXT STEPS:
- Integrate these features with traditional credit bureau data
- Create composite scores combining e-commerce and financial data
- Develop models using both traditional and alternative data sources
""")

# Save the enhanced features for potential model integration
output_path = "../data/processed/ecommerce_features.csv"
ecommerce_features.to_csv(output_path, index=False)
print(f"\nE-commerce features saved to: {output_path}")

# Telecom Features Prototype

This section demonstrates how to engineer telecom features that indicate customer stability and usage consistency for credit risk assessment.

## Telecom Data Analysis and Feature Engineering

We'll create features from telecom data including:
- **Customer Tenure**: Length of relationship with telecom provider
- **Usage Consistency**: Stability in calls, SMS, and data usage patterns
- **Service Utilization**: Pattern analysis of different telecom services
- **Payment Behavior**: Bill payment consistency and patterns

In [ ]:
# Load or create telecom churn dataset
def create_telecom_dataset():
    """
    Create a comprehensive telecom dataset for feature engineering.
    This simulates a realistic telecom churn dataset with customer usage patterns.
    """
    np.random.seed(42)
    n_customers = 2000
    
    # Generate customer demographics and tenure
    customer_ids = range(1, n_customers + 1)
    tenure_months = np.random.exponential(24, n_customers).astype(int)  # Average 24 months
    tenure_months = np.clip(tenure_months, 1, 120)  # 1 month to 10 years
    
    # Generate monthly usage data for multiple months (last 12 months)
    monthly_data = []
    
    for customer_id in customer_ids:
        customer_tenure = tenure_months[customer_id - 1]
        months_to_generate = min(12, customer_tenure)  # Last 12 months or tenure
        
        # Customer behavior profile (stable vs volatile)
        is_stable_customer = np.random.choice([True, False], p=[0.7, 0.3])
        
        for month in range(months_to_generate):
            # Base usage patterns
            base_calls = np.random.poisson(100) if is_stable_customer else np.random.poisson(80)
            base_sms = np.random.poisson(50) if is_stable_customer else np.random.poisson(40)
            base_data_gb = np.random.exponential(5) if is_stable_customer else np.random.exponential(4)
            
            # Add variability (stable customers have less variance)
            variance_factor = 0.2 if is_stable_customer else 0.6
            
            calls_made = max(0, base_calls + np.random.normal(0, base_calls * variance_factor))
            sms_sent = max(0, base_sms + np.random.normal(0, base_sms * variance_factor))
            data_used_gb = max(0, base_data_gb + np.random.normal(0, base_data_gb * variance_factor))
            
            # Calculate bill amount based on usage
            call_charges = calls_made * 0.02  # $0.02 per call
            sms_charges = sms_sent * 0.01     # $0.01 per SMS
            data_charges = data_used_gb * 2   # $2 per GB
            base_plan = 25                    # $25 base plan
            
            monthly_bill = base_plan + call_charges + sms_charges + data_charges
            
            # Payment behavior (stable customers pay on time more often)
            payment_delay_days = 0
            if not is_stable_customer and np.random.random() < 0.3:
                payment_delay_days = np.random.poisson(5)
            
            monthly_data.append({
                'customer_id': customer_id,
                'month': month + 1,
                'calls_made': int(calls_made),
                'sms_sent': int(sms_sent),
                'data_used_gb': round(data_used_gb, 2),
                'monthly_bill': round(monthly_bill, 2),
                'payment_delay_days': payment_delay_days,
                'is_stable_profile': is_stable_customer
            })
    
    monthly_df = pd.DataFrame(monthly_data)
    
    # Create customer summary dataset
    customer_data = []
    for customer_id in customer_ids:
        tenure = tenure_months[customer_id - 1]
        
        # Customer monthly income (affects ability to pay)
        monthly_income = np.random.normal(3000, 1000)
        monthly_income = max(1000, monthly_income)  # Minimum $1000
        
        # Service plan type
        plan_types = ['Basic', 'Standard', 'Premium']
        plan_weights = [0.4, 0.4, 0.2]
        plan_type = np.random.choice(plan_types, p=plan_weights)
        
        # Internet service type
        internet_types = ['DSL', 'Fiber', 'Cable', 'None']
        internet_weights = [0.3, 0.3, 0.3, 0.1]
        internet_service = np.random.choice(internet_types, p=internet_weights)
        
        # Additional services
        has_streaming = np.random.choice([True, False], p=[0.6, 0.4])
        has_cloud_backup = np.random.choice([True, False], p=[0.3, 0.7])
        has_tech_support = np.random.choice([True, False], p=[0.2, 0.8])
        
        # Contract type
        contract_types = ['Month-to-month', 'One year', 'Two year']
        contract_weights = [0.5, 0.3, 0.2]
        contract_type = np.random.choice(contract_types, p=contract_weights)
        
        # Customer satisfaction and churn
        # Longer tenure and stable usage = less likely to churn
        churn_probability = max(0.05, 0.4 - (tenure / 120) * 0.3)
        churned = np.random.choice([True, False], p=[churn_probability, 1 - churn_probability])
        
        customer_data.append({
            'customer_id': customer_id,
            'tenure_months': tenure,
            'monthly_income': round(monthly_income, 2),
            'plan_type': plan_type,
            'internet_service': internet_service,
            'has_streaming': has_streaming,
            'has_cloud_backup': has_cloud_backup,
            'has_tech_support': has_tech_support,
            'contract_type': contract_type,
            'churned': churned
        })
    
    customer_df = pd.DataFrame(customer_data)
    
    return customer_df, monthly_df

# Create the telecom datasets
print("Creating comprehensive telecom dataset...")
telecom_customers, telecom_monthly = create_telecom_dataset()

print(f"Customer data shape: {telecom_customers.shape}")
print(f"Monthly usage data shape: {telecom_monthly.shape}")

print("\nCustomer Data Sample:")
display(telecom_customers.head())

print("\nMonthly Usage Data Sample:")
display(telecom_monthly.head())

print("\nDataset Summary:")
print(f"Total customers: {telecom_customers['customer_id'].nunique()}")
print(f"Average tenure: {telecom_customers['tenure_months'].mean():.1f} months")
print(f"Churn rate: {telecom_customers['churned'].mean():.2%}")
print(f"Total monthly records: {len(telecom_monthly)}")

# Basic statistics
print("\nUsage Statistics:")
print(telecom_monthly[['calls_made', 'sms_sent', 'data_used_gb', 'monthly_bill']].describe())

In [ ]:
# Engineer stability features from telecom data
def create_telecom_stability_features(customer_df, monthly_df):
    """
    Create stability and consistency features from telecom data that indicate creditworthiness.
    """
    
    # 1. TENURE-BASED STABILITY FEATURES
    stability_features = customer_df[['customer_id', 'tenure_months']].copy()
    
    # Convert tenure to years and create stability categories
    stability_features['tenure_in_years'] = stability_features['tenure_months'] / 12
    
    def categorize_tenure_stability(tenure_months):
        if tenure_months < 6:
            return 'New_Customer'
        elif tenure_months < 12:
            return 'Short_Term'
        elif tenure_months < 24:
            return 'Medium_Term'
        elif tenure_months < 48:
            return 'Long_Term'
        else:
            return 'Very_Stable'
    
    stability_features['tenure_category'] = stability_features['tenure_months'].apply(categorize_tenure_stability)
    
    # 2. USAGE CONSISTENCY FEATURES
    # Calculate monthly usage statistics for each customer
    usage_consistency = monthly_df.groupby('customer_id').agg({
        'calls_made': ['mean', 'std', 'min', 'max', 'count'],
        'sms_sent': ['mean', 'std', 'min', 'max'],
        'data_used_gb': ['mean', 'std', 'min', 'max'],
        'monthly_bill': ['mean', 'std', 'min', 'max']
    }).round(2)
    
    # Flatten column names
    usage_consistency.columns = [f"{col[0]}_{col[1]}" for col in usage_consistency.columns]
    usage_consistency = usage_consistency.reset_index()
    
    # Calculate coefficient of variation (CV) for usage consistency
    # Lower CV = more consistent usage = higher stability
    usage_consistency['calls_consistency_cv'] = usage_consistency['calls_made_std'] / usage_consistency['calls_made_mean']
    usage_consistency['sms_consistency_cv'] = usage_consistency['sms_sent_std'] / usage_consistency['sms_sent_mean']
    usage_consistency['data_consistency_cv'] = usage_consistency['data_used_gb_std'] / usage_consistency['data_used_gb_mean']
    usage_consistency['bill_consistency_cv'] = usage_consistency['monthly_bill_std'] / usage_consistency['monthly_bill_mean']
    
    # Replace inf and NaN values
    consistency_cols = ['calls_consistency_cv', 'sms_consistency_cv', 'data_consistency_cv', 'bill_consistency_cv']
    for col in consistency_cols:
        usage_consistency[col] = usage_consistency[col].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # Create overall usage consistency score (lower is better)
    usage_consistency['overall_usage_consistency'] = usage_consistency[consistency_cols].mean(axis=1)
    
    # 3. PAYMENT BEHAVIOR FEATURES
    payment_behavior = monthly_df.groupby('customer_id').agg({
        'payment_delay_days': ['mean', 'std', 'max', 'sum'],
        'month': 'count'  # number of months of data
    }).round(2)
    
    payment_behavior.columns = [f"payment_{col[0]}_{col[1]}" if col[0] != 'month' else 'months_of_data' 
                               for col in payment_behavior.columns]
    payment_behavior = payment_behavior.reset_index()
    
    # Calculate payment reliability score
    payment_behavior['payment_reliability_score'] = np.where(
        payment_behavior['payment_payment_delay_days_mean'] == 0, 100,
        100 - (payment_behavior['payment_payment_delay_days_mean'] * 10)  # Penalty for delays
    )
    payment_behavior['payment_reliability_score'] = np.clip(payment_behavior['payment_reliability_score'], 0, 100)
    
    # 4. SERVICE UTILIZATION FEATURES
    service_features = customer_df[['customer_id', 'plan_type', 'internet_service', 
                                   'has_streaming', 'has_cloud_backup', 'has_tech_support', 
                                   'contract_type']].copy()
    
    # Count of additional services (indicates engagement)
    service_features['additional_services_count'] = (
        service_features['has_streaming'].astype(int) +
        service_features['has_cloud_backup'].astype(int) +
        service_features['has_tech_support'].astype(int)
    )
    
    # Contract stability score
    contract_stability_map = {
        'Month-to-month': 1,
        'One year': 2,
        'Two year': 3
    }
    service_features['contract_stability_score'] = service_features['contract_type'].map(contract_stability_map)
    
    # 5. SPENDING BEHAVIOR FEATURES
    spending_features = monthly_df.groupby('customer_id').agg({
        'monthly_bill': ['mean', 'std', 'min', 'max', 'sum']
    }).round(2)
    
    spending_features.columns = [f"spending_{col[1]}" for col in spending_features.columns]
    spending_features = spending_features.reset_index()
    
    # Calculate spending stability
    spending_features['spending_stability'] = 1 / (1 + spending_features['spending_std'])  # Higher = more stable
    
    # Merge all features
    telecom_features = stability_features
    feature_dfs = [usage_consistency, payment_behavior, service_features, spending_features]
    
    for df in feature_dfs:
        telecom_features = telecom_features.merge(df, on='customer_id', how='left')
    
    return telecom_features

# Create telecom stability features
print("Engineering telecom stability and consistency features...")
telecom_features = create_telecom_stability_features(telecom_customers, telecom_monthly)

print(f"Telecom features shape: {telecom_features.shape}")
print(f"Features created: {telecom_features.columns.tolist()}")

# Display key stability metrics
print("\nTenure Stability Distribution:")
print(telecom_features['tenure_category'].value_counts())

print("\nKey Stability Features Summary:")
stability_cols = ['tenure_in_years', 'overall_usage_consistency', 'payment_reliability_score', 
                 'contract_stability_score', 'spending_stability']

for col in stability_cols:
    if col in telecom_features.columns:
        print(f"\n{col}:")
        print(f"  Mean: {telecom_features[col].mean():.3f}")
        print(f"  Std:  {telecom_features[col].std():.3f}")
        print(f"  Min:  {telecom_features[col].min():.3f}")
        print(f"  Max:  {telecom_features[col].max():.3f}")

display(telecom_features.head())

In [ ]:
# Create advanced usage consistency and behavioral features
def create_advanced_telecom_features(monthly_df):
    """
    Create advanced behavioral features from telecom usage patterns.
    """
    
    # 1. TEMPORAL USAGE PATTERNS
    temporal_features = []
    
    for customer_id in monthly_df['customer_id'].unique():
        customer_data = monthly_df[monthly_df['customer_id'] == customer_id].sort_values('month')
        
        if len(customer_data) < 3:  # Need at least 3 months for trend analysis
            continue
            
        features = {'customer_id': customer_id}
        
        # Usage trend analysis (increasing, stable, decreasing)
        for usage_col in ['calls_made', 'sms_sent', 'data_used_gb']:
            usage_values = customer_data[usage_col].values
            
            # Calculate trend slope
            months = np.arange(len(usage_values))
            if len(months) > 1:
                slope, intercept = np.polyfit(months, usage_values, 1)
                features[f'{usage_col}_trend_slope'] = slope
                
                # Calculate trend consistency (R-squared)
                predicted = slope * months + intercept
                ss_res = np.sum((usage_values - predicted) ** 2)
                ss_tot = np.sum((usage_values - np.mean(usage_values)) ** 2)
                r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
                features[f'{usage_col}_trend_consistency'] = r_squared
            else:
                features[f'{usage_col}_trend_slope'] = 0
                features[f'{usage_col}_trend_consistency'] = 0
        
        # 2. USAGE VOLATILITY FEATURES
        for usage_col in ['calls_made', 'sms_sent', 'data_used_gb', 'monthly_bill']:
            usage_values = customer_data[usage_col].values
            
            if len(usage_values) > 1:
                # Month-over-month changes
                changes = np.diff(usage_values)
                features[f'{usage_col}_volatility'] = np.std(changes) if len(changes) > 0 else 0
                
                # Percentage changes
                pct_changes = []
                for i in range(1, len(usage_values)):
                    if usage_values[i-1] != 0:
                        pct_change = (usage_values[i] - usage_values[i-1]) / usage_values[i-1] * 100
                        pct_changes.append(abs(pct_change))
                
                features[f'{usage_col}_pct_volatility'] = np.mean(pct_changes) if pct_changes else 0
            else:
                features[f'{usage_col}_volatility'] = 0
                features[f'{usage_col}_pct_volatility'] = 0
        
        # 3. PAYMENT BEHAVIOR PATTERNS
        payment_delays = customer_data['payment_delay_days'].values
        features['payment_delay_frequency'] = (payment_delays > 0).sum() / len(payment_delays)
        features['max_consecutive_delays'] = 0
        
        # Calculate max consecutive payment delays
        current_streak = 0
        max_streak = 0
        for delay in payment_delays:
            if delay > 0:
                current_streak += 1
                max_streak = max(max_streak, current_streak)
            else:
                current_streak = 0
        features['max_consecutive_delays'] = max_streak
        
        # 4. SERVICE USAGE EFFICIENCY
        total_bill = customer_data['monthly_bill'].sum()
        total_usage = (
            customer_data['calls_made'].sum() * 0.02 +  # Call value
            customer_data['sms_sent'].sum() * 0.01 +    # SMS value
            customer_data['data_used_gb'].sum() * 2     # Data value
        )
        
        features['usage_efficiency_ratio'] = total_usage / total_bill if total_bill > 0 else 0
        
        temporal_features.append(features)
    
    return pd.DataFrame(temporal_features)

# Create advanced features
print("Creating advanced telecom behavioral features...")
advanced_features = create_advanced_telecom_features(telecom_monthly)

print(f"Advanced features shape: {advanced_features.shape}")
print(f"Advanced features: {[col for col in advanced_features.columns if col != 'customer_id']}")

# Merge with main telecom features
telecom_features_enhanced = telecom_features.merge(advanced_features, on='customer_id', how='left')

print(f"Enhanced telecom features shape: {telecom_features_enhanced.shape}")

# Fill NaN values for customers with insufficient data
feature_cols = [col for col in advanced_features.columns if col != 'customer_id']
telecom_features_enhanced[feature_cols] = telecom_features_enhanced[feature_cols].fillna(0)

# Display sample of advanced features
print("\nAdvanced Features Sample:")
advanced_sample_cols = ['customer_id', 'calls_made_trend_slope', 'data_used_gb_volatility', 
                       'payment_delay_frequency', 'usage_efficiency_ratio']
display(telecom_features_enhanced[advanced_sample_cols].head(10))

print("\nAdvanced Features Statistics:")
for col in ['calls_made_trend_slope', 'data_used_gb_volatility', 'payment_delay_frequency', 'usage_efficiency_ratio']:
    if col in telecom_features_enhanced.columns:
        print(f"\n{col}:")
        print(f"  Mean: {telecom_features_enhanced[col].mean():.4f}")
        print(f"  Std:  {telecom_features_enhanced[col].std():.4f}")
        print(f"  Min:  {telecom_features_enhanced[col].min():.4f}")
        print(f"  Max:  {telecom_features_enhanced[col].max():.4f}")

In [ ]:
# Visualize telecom features and their relationship to churn/stability
# Merge with churn information for analysis
telecom_analysis = telecom_features_enhanced.merge(
    telecom_customers[['customer_id', 'churned']], 
    on='customer_id', 
    how='left'
)

# Create comprehensive visualizations
fig, axes = plt.subplots(4, 3, figsize=(20, 16))
fig.suptitle('Telecom Features Analysis for Credit Risk Assessment', fontsize=16, fontweight='bold')

# 1. Tenure Analysis
axes[0, 0].hist(telecom_analysis['tenure_in_years'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Customer Tenure Distribution')
axes[0, 0].set_xlabel('Tenure (Years)')
axes[0, 0].set_ylabel('Number of Customers')

# 2. Usage Consistency vs Churn
churn_consistency = telecom_analysis.groupby('churned')['overall_usage_consistency'].mean()
axes[0, 1].bar(['Retained', 'Churned'], churn_consistency.values, color=['green', 'red'], alpha=0.7)
axes[0, 1].set_title('Usage Consistency by Churn Status')
axes[0, 1].set_ylabel('Average Usage Consistency')

# 3. Payment Reliability vs Churn
churn_payment = telecom_analysis.groupby('churned')['payment_reliability_score'].mean()
axes[0, 2].bar(['Retained', 'Churned'], churn_payment.values, color=['green', 'red'], alpha=0.7)
axes[0, 2].set_title('Payment Reliability by Churn Status')
axes[0, 2].set_ylabel('Average Payment Reliability Score')

# 4. Tenure Category Distribution
tenure_counts = telecom_analysis['tenure_category'].value_counts()
axes[1, 0].pie(tenure_counts.values, labels=tenure_counts.index, autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Customer Tenure Categories')

# 5. Contract Stability vs Churn
contract_churn = telecom_analysis.groupby('contract_type')['churned'].mean()
axes[1, 1].bar(contract_churn.index, contract_churn.values, color='orange', alpha=0.7)
axes[1, 1].set_title('Churn Rate by Contract Type')
axes[1, 1].set_ylabel('Churn Rate')
axes[1, 1].tick_params(axis='x', rotation=45)

# 6. Data Usage Volatility Distribution
axes[1, 2].hist(telecom_analysis['data_used_gb_volatility'], bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1, 2].set_title('Data Usage Volatility Distribution')
axes[1, 2].set_xlabel('Data Usage Volatility')
axes[1, 2].set_ylabel('Number of Customers')

# 7. Monthly Bill Consistency
axes[2, 0].scatter(telecom_analysis['spending_mean'], telecom_analysis['spending_stability'], 
                  c=telecom_analysis['churned'], cmap='RdYlGn_r', alpha=0.6)
axes[2, 0].set_title('Bill Amount vs Spending Stability')
axes[2, 0].set_xlabel('Average Monthly Bill')
axes[2, 0].set_ylabel('Spending Stability')

# 8. Additional Services vs Churn
service_churn = telecom_analysis.groupby('additional_services_count')['churned'].mean()
axes[2, 1].bar(service_churn.index, service_churn.values, color='cyan', alpha=0.7)
axes[2, 1].set_title('Churn Rate by Number of Additional Services')
axes[2, 1].set_xlabel('Number of Additional Services')
axes[2, 1].set_ylabel('Churn Rate')

# 9. Payment Delay Frequency Distribution
axes[2, 2].hist(telecom_analysis['payment_delay_frequency'], bins=20, alpha=0.7, color='red', edgecolor='black')
axes[2, 2].set_title('Payment Delay Frequency Distribution')
axes[2, 2].set_xlabel('Payment Delay Frequency')
axes[2, 2].set_ylabel('Number of Customers')

# 10. Usage Trend Analysis
axes[3, 0].hist(telecom_analysis['calls_made_trend_slope'], bins=30, alpha=0.7, color='gold', edgecolor='black')
axes[3, 0].set_title('Call Usage Trend Distribution')
axes[3, 0].set_xlabel('Calls Trend Slope')
axes[3, 0].set_ylabel('Number of Customers')

# 11. Correlation heatmap of key features
key_telecom_features = [
    'tenure_in_years', 'overall_usage_consistency', 'payment_reliability_score',
    'contract_stability_score', 'spending_stability', 'data_used_gb_volatility',
    'payment_delay_frequency', 'additional_services_count'
]

correlation_matrix = telecom_analysis[key_telecom_features].corr()
im = axes[3, 1].imshow(correlation_matrix, cmap='coolwarm', aspect='auto')
axes[3, 1].set_xticks(range(len(key_telecom_features)))
axes[3, 1].set_yticks(range(len(key_telecom_features)))
axes[3, 1].set_xticklabels([f.replace('_', '\n') for f in key_telecom_features], rotation=45, ha='right')
axes[3, 1].set_yticklabels([f.replace('_', '\n') for f in key_telecom_features])
axes[3, 1].set_title('Feature Correlation Matrix')

# Add correlation values
for i in range(len(key_telecom_features)):
    for j in range(len(key_telecom_features)):
        axes[3, 1].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}', 
                       ha='center', va='center', fontsize=8)

# 12. Risk Score Distribution
# Create a simple risk score based on key features
telecom_analysis['telecom_risk_score'] = (
    (5 - telecom_analysis['tenure_in_years'].clip(0, 5)) * 20 +  # Tenure risk (shorter = higher risk)
    telecom_analysis['overall_usage_consistency'] * 30 +         # Usage inconsistency risk
    (100 - telecom_analysis['payment_reliability_score']) * 0.3 + # Payment risk
    telecom_analysis['payment_delay_frequency'] * 50            # Payment delay risk
)

axes[3, 2].hist(telecom_analysis['telecom_risk_score'], bins=25, alpha=0.7, color='darkred', edgecolor='black')
axes[3, 2].set_title('Telecom Risk Score Distribution')
axes[3, 2].set_xlabel('Risk Score (Higher = More Risk)')
axes[3, 2].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

# Feature importance analysis
print("\n" + "="*70)
print("TELECOM FEATURES CREDIT RISK INSIGHTS")
print("="*70)

print(f"""
KEY STABILITY INDICATORS:

1. CUSTOMER TENURE:
   • Average tenure: {telecom_analysis['tenure_in_years'].mean():.1f} years
   • Customers with >2 years: {(telecom_analysis['tenure_in_years'] > 2).sum()} ({(telecom_analysis['tenure_in_years'] > 2).mean():.1%})
   • Churn rate by tenure category:
""")

for category in telecom_analysis['tenure_category'].unique():
    if pd.notna(category):
        churn_rate = telecom_analysis[telecom_analysis['tenure_category'] == category]['churned'].mean()
        print(f"     - {category}: {churn_rate:.1%}")

print(f"""
2. USAGE CONSISTENCY:
   • High consistency customers: {(telecom_analysis['overall_usage_consistency'] < 0.3).sum()} ({(telecom_analysis['overall_usage_consistency'] < 0.3).mean():.1%})
   • Average usage consistency: {telecom_analysis['overall_usage_consistency'].mean():.3f}

3. PAYMENT BEHAVIOR:
   • Perfect payment customers: {(telecom_analysis['payment_reliability_score'] == 100).sum()} ({(telecom_analysis['payment_reliability_score'] == 100).mean():.1%})
   • Average payment reliability: {telecom_analysis['payment_reliability_score'].mean():.1f}/100

4. CONTRACT STABILITY:
   • Long-term contracts: {(telecom_analysis['contract_stability_score'] >= 2).sum()} ({(telecom_analysis['contract_stability_score'] >= 2).mean():.1%})
""")

print("\nCREDIT RISK IMPLICATIONS:")
print("""
• TENURE → Income Stability: Longer telecom relationships indicate stable residence and income
• USAGE CONSISTENCY → Financial Predictability: Consistent usage patterns suggest regular income
• PAYMENT RELIABILITY → Credit Discipline: On-time telecom payments predict loan payment behavior
• CONTRACT TYPE → Financial Commitment: Long-term contracts show planning capability
• SERVICE UTILIZATION → Financial Capacity: Multiple services indicate disposable income
""")

# Save telecom features
telecom_output_path = "../data/processed/telecom_features.csv"
telecom_analysis.to_csv(telecom_output_path, index=False)
print(f"\nTelecom features saved to: {telecom_output_path}")

In [ ]:
# Demonstrate integration of telecom features with credit risk assessment
def create_telecom_credit_features(telecom_df):
    """
    Create credit-specific features from telecom data for risk assessment.
    """
    
    # Create a copy for feature engineering
    credit_features = telecom_df.copy()
    
    # 1. STABILITY SCORE (0-100, higher is better)
    credit_features['telecom_stability_score'] = (
        # Tenure component (40% weight)
        (credit_features['tenure_in_years'].clip(0, 5) / 5 * 40) +
        
        # Usage consistency component (30% weight) - lower consistency CV is better
        ((1 - credit_features['overall_usage_consistency'].clip(0, 1)) * 30) +
        
        # Payment reliability component (30% weight)
        (credit_features['payment_reliability_score'] / 100 * 30)
    ).round(1)
    
    # 2. FINANCIAL CAPACITY INDICATORS
    # Monthly spending capacity
    credit_features['monthly_telecom_spending'] = credit_features['spending_mean']
    
    # Spending growth trend (positive = increasing capacity)
    credit_features['spending_growth_indicator'] = np.where(
        credit_features['monthly_bill_trend_slope'] > 0, 1, 0
    )
    
    # Service adoption score (indicates financial capacity)
    credit_features['service_adoption_score'] = (
        credit_features['additional_services_count'] * 25 +  # Each service worth 25 points
        credit_features['contract_stability_score'] * 10     # Contract stability worth 10 points
    ).clip(0, 100)
    
    # 3. BEHAVIORAL RISK INDICATORS
    # Payment discipline score
    credit_features['payment_discipline_score'] = (
        100 - (credit_features['payment_delay_frequency'] * 100)
    ).clip(0, 100)
    
    # Usage volatility risk (higher volatility = higher risk)
    max_volatility = credit_features[['calls_made_volatility', 'sms_sent_volatility', 'data_used_gb_volatility']].max(axis=1)
    credit_features['usage_volatility_risk'] = (max_volatility / credit_features[['calls_made_volatility', 'sms_sent_volatility', 'data_used_gb_volatility']].max().max() * 100).fillna(0)
    
    # 4. OVERALL TELECOM CREDIT SCORE
    credit_features['telecom_credit_score'] = (
        credit_features['telecom_stability_score'] * 0.4 +      # 40% stability
        credit_features['payment_discipline_score'] * 0.3 +     # 30% payment behavior
        credit_features['service_adoption_score'] * 0.2 +       # 20% financial capacity
        (100 - credit_features['usage_volatility_risk']) * 0.1  # 10% consistency
    ).round(1)
    
    # 5. RISK CATEGORIES
    def categorize_telecom_risk(score):
        if score >= 80:
            return 'Low_Risk'
        elif score >= 60:
            return 'Medium_Risk'
        elif score >= 40:
            return 'High_Risk'
        else:
            return 'Very_High_Risk'
    
    credit_features['telecom_risk_category'] = credit_features['telecom_credit_score'].apply(categorize_telecom_risk)
    
    return credit_features

# Apply credit-specific feature engineering
print("Creating telecom-based credit risk features...")
telecom_credit_features = create_telecom_credit_features(telecom_analysis)

print(f"Telecom credit features shape: {telecom_credit_features.shape}")

# Analyze credit risk distribution
print("\nTelecom Credit Risk Distribution:")
print(telecom_credit_features['telecom_risk_category'].value_counts())

print("\nTelecom Credit Score Statistics:")
print(telecom_credit_features['telecom_credit_score'].describe())

# Correlation with churn (proxy for credit risk)
print(f"\nCorrelation between Telecom Credit Score and Churn: {telecom_credit_features['telecom_credit_score'].corr(telecom_credit_features['churned']):.4f}")

# Feature importance for credit assessment
credit_risk_features = [
    'telecom_stability_score', 'payment_discipline_score', 
    'service_adoption_score', 'usage_volatility_risk', 'telecom_credit_score'
]

print("\nCredit Risk Feature Analysis:")
for feature in credit_risk_features:
    low_risk_avg = telecom_credit_features[telecom_credit_features['churned'] == False][feature].mean()
    high_risk_avg = telecom_credit_features[telecom_credit_features['churned'] == True][feature].mean()
    print(f"{feature}:")
    print(f"  Low Risk (Retained): {low_risk_avg:.2f}")
    print(f"  High Risk (Churned): {high_risk_avg:.2f}")
    print(f"  Difference: {low_risk_avg - high_risk_avg:.2f}")
    print()

# Visualization of credit risk features
plt.figure(figsize=(15, 10))

# Credit score distribution by risk category
plt.subplot(2, 3, 1)
for category in telecom_credit_features['telecom_risk_category'].unique():
    data = telecom_credit_features[telecom_credit_features['telecom_risk_category'] == category]['telecom_credit_score']
    plt.hist(data, alpha=0.6, label=category, bins=15)
plt.title('Credit Score Distribution by Risk Category')
plt.xlabel('Telecom Credit Score')
plt.ylabel('Number of Customers')
plt.legend()

# Stability vs Payment Discipline
plt.subplot(2, 3, 2)
scatter = plt.scatter(telecom_credit_features['telecom_stability_score'], 
                     telecom_credit_features['payment_discipline_score'],
                     c=telecom_credit_features['churned'], cmap='RdYlGn_r', alpha=0.6)
plt.xlabel('Telecom Stability Score')
plt.ylabel('Payment Discipline Score')
plt.title('Stability vs Payment Discipline')
plt.colorbar(scatter, label='Churned')

# Risk category vs actual churn
plt.subplot(2, 3, 3)
churn_by_risk = telecom_credit_features.groupby('telecom_risk_category')['churned'].mean()
plt.bar(churn_by_risk.index, churn_by_risk.values, color=['green', 'yellow', 'orange', 'red'])
plt.title('Actual Churn Rate by Telecom Risk Category')
plt.ylabel('Churn Rate')
plt.xticks(rotation=45)

# Service adoption vs credit score
plt.subplot(2, 3, 4)
plt.scatter(telecom_credit_features['service_adoption_score'], 
           telecom_credit_features['telecom_credit_score'], alpha=0.6)
plt.xlabel('Service Adoption Score')
plt.ylabel('Telecom Credit Score')
plt.title('Service Adoption vs Credit Score')

# Tenure vs credit score
plt.subplot(2, 3, 5)
plt.scatter(telecom_credit_features['tenure_in_years'], 
           telecom_credit_features['telecom_credit_score'], alpha=0.6)
plt.xlabel('Tenure (Years)')
plt.ylabel('Telecom Credit Score')
plt.title('Tenure vs Credit Score')

# Feature importance comparison
plt.subplot(2, 3, 6)
feature_importance = []
for feature in ['telecom_stability_score', 'payment_discipline_score', 'service_adoption_score']:
    corr_with_score = abs(telecom_credit_features[feature].corr(telecom_credit_features['telecom_credit_score']))
    feature_importance.append(corr_with_score)

plt.bar(['Stability', 'Payment', 'Service'], feature_importance, color=['blue', 'green', 'purple'])
plt.title('Feature Importance for Credit Score')
plt.ylabel('Correlation with Credit Score')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("TELECOM FEATURES FOR CREDIT RISK ASSESSMENT - SUMMARY")
print("="*80)

print(f"""
FEATURE ENGINEERING COMPLETE:

1. STABILITY FEATURES:
   ✓ Customer Tenure ({telecom_credit_features['tenure_in_years'].mean():.1f} years average)
   ✓ Usage Consistency (CV-based measurement)
   ✓ Service Commitment (contract types and additional services)

2. PAYMENT BEHAVIOR FEATURES:
   ✓ Payment Reliability Score ({telecom_credit_features['payment_reliability_score'].mean():.1f}/100 average)
   ✓ Payment Discipline Score ({telecom_credit_features['payment_discipline_score'].mean():.1f}/100 average)
   ✓ Payment Delay Patterns

3. FINANCIAL CAPACITY INDICATORS:
   ✓ Monthly Spending Patterns (${telecom_credit_features['monthly_telecom_spending'].mean():.2f} average)
   ✓ Service Adoption Score ({telecom_credit_features['service_adoption_score'].mean():.1f}/100 average)
   ✓ Spending Growth Trends

4. RISK ASSESSMENT:
   ✓ Comprehensive Telecom Credit Score ({telecom_credit_features['telecom_credit_score'].mean():.1f}/100 average)
   ✓ Risk Categorization (Low/Medium/High/Very High)
   ✓ Correlation with churn behavior: {telecom_credit_features['telecom_credit_score'].corr(telecom_credit_features['churned']):.3f}

CREDIT RISK INSIGHTS:
• Longer tenure customers show {(1-telecom_credit_features[telecom_credit_features['tenure_in_years'] > 2]['churned'].mean()):.1%} retention
• Consistent usage patterns reduce risk by {((telecom_credit_features[telecom_credit_features['overall_usage_consistency'] < 0.3]['churned'].mean() - telecom_credit_features['churned'].mean()) * -100):.1f} percentage points
• Perfect payment history customers have {(1-telecom_credit_features[telecom_credit_features['payment_reliability_score'] == 100]['churned'].mean()):.1%} retention rate

INTEGRATION READY: These features can now be integrated with traditional credit data for enhanced risk assessment.
""")

# Save final telecom credit features
final_output_path = "../data/processed/telecom_credit_features.csv"
telecom_credit_features.to_csv(final_output_path, index=False)
print(f"\nFinal telecom credit features saved to: {final_output_path}")

# 🗺️ Mobility Features - GPS Data Analysis

Mobility patterns can provide valuable insights for credit risk assessment:
- **Commute stability**: Regular work-home patterns indicate employment stability
- **Location consistency**: Frequent locations suggest residential and employment stability
- **Travel patterns**: Distance between work/home can indicate income potential and lifestyle

We'll prototype features using GPS trajectory data to identify:
1. Work location (most frequent daytime location)
2. Home location (most frequent nighttime location)  
3. Commute distance and patterns
4. Location stability metrics

In [ ]:
# Import libraries for GPS data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, time
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Since we don't have access to the actual Geolife dataset, we'll simulate realistic GPS trajectory data
# This simulates a user's GPS data over 30 days with realistic movement patterns

np.random.seed(42)

# Define realistic locations (Beijing coordinates similar to Geolife dataset)
home_location = [39.9042, 116.4074]  # Base home location
work_location = [39.9388, 116.3974]  # Base work location (about 4km from home)

# Generate 30 days of GPS trajectories
start_date = pd.Timestamp('2024-01-01')
gps_data = []

for day in range(30):
    current_date = start_date + pd.Timedelta(days=day)
    
    # Generate hourly GPS points for each day
    for hour in range(24):
        timestamp = current_date + pd.Timedelta(hours=hour, minutes=np.random.randint(0, 60))
        
        # Determine primary location based on time of day
        if 22 <= hour or hour <= 6:  # Nighttime hours (10pm - 6am)
            # At home with small random variations
            lat = home_location[0] + np.random.normal(0, 0.002)
            lon = home_location[1] + np.random.normal(0, 0.002)
            location_type = 'home'
        elif 9 <= hour <= 17:  # Daytime work hours (9am - 5pm)
            # At work with small random variations
            lat = work_location[0] + np.random.normal(0, 0.001)
            lon = work_location[1] + np.random.normal(0, 0.001)
            location_type = 'work'
        else:  # Commute or other activities
            # Random locations between home and work, or nearby areas
            if np.random.random() < 0.7:  # 70% chance of being in transit
                # Linear interpolation between home and work with noise
                ratio = np.random.random()
                lat = home_location[0] * (1-ratio) + work_location[0] * ratio + np.random.normal(0, 0.003)
                lon = home_location[1] * (1-ratio) + work_location[1] * ratio + np.random.normal(0, 0.003)
                location_type = 'transit'
            else:  # Other locations (shopping, dining, etc.)
                lat = home_location[0] + np.random.normal(0, 0.01)
                lon = home_location[1] + np.random.normal(0, 0.01)
                location_type = 'other'
        
        gps_data.append({
            'timestamp': timestamp,
            'latitude': lat,
            'longitude': lon,
            'hour': hour,
            'location_type': location_type  # This is just for our reference, not normally available
        })

# Create DataFrame
gps_df = pd.DataFrame(gps_data)
gps_df['date'] = gps_df['timestamp'].dt.date
gps_df['time'] = gps_df['timestamp'].dt.time

print(f"📊 Generated {len(gps_df)} GPS trajectory points over {gps_df['date'].nunique()} days")
print(f"📍 Latitude range: {gps_df['latitude'].min():.4f} to {gps_df['latitude'].max():.4f}")
print(f"📍 Longitude range: {gps_df['longitude'].min():.4f} to {gps_df['longitude'].max():.4f}")

gps_df.head(10)

In [ ]:
# 1. Identify Work Location (Most frequent location during daytime hours 9am-5pm)

# Filter GPS data for daytime work hours (9am - 5pm)
work_hours_data = gps_df[(gps_df['hour'] >= 9) & (gps_df['hour'] <= 17)].copy()

print(f"📊 Analyzing {len(work_hours_data)} GPS points during work hours (9am-5pm)")

# Use DBSCAN clustering to find location clusters during work hours
work_coords = work_hours_data[['latitude', 'longitude']].values

# DBSCAN parameters: eps in degrees (roughly 100 meters), min_samples for cluster
eps_degrees = 0.001  # Approximately 100 meters at Beijing latitude
min_samples = 5     # Minimum points to form a cluster

dbscan_work = DBSCAN(eps=eps_degrees, min_samples=min_samples)
work_clusters = dbscan_work.fit_predict(work_coords)

work_hours_data['cluster'] = work_clusters

# Find the most frequent cluster (excluding noise points labeled as -1)
cluster_counts = pd.Series(work_clusters).value_counts()
cluster_counts = cluster_counts[cluster_counts.index != -1]  # Remove noise

if len(cluster_counts) > 0:
    most_frequent_work_cluster = cluster_counts.index[0]
    work_cluster_data = work_hours_data[work_hours_data['cluster'] == most_frequent_work_cluster]
    
    # Calculate centroid of the most frequent work cluster
    work_location_inferred = [
        work_cluster_data['latitude'].mean(),
        work_cluster_data['longitude'].mean()
    ]
    
    print(f"🏢 Inferred WORK location: ({work_location_inferred[0]:.6f}, {work_location_inferred[1]:.6f})")
    print(f"📍 Based on {len(work_cluster_data)} GPS points in largest cluster")
    print(f"🕒 Time range: {work_cluster_data['timestamp'].min()} to {work_cluster_data['timestamp'].max()}")
    
    # Show cluster statistics
    print(f"\n📊 Work Hours Clustering Results:")
    print(f"   - Total clusters found: {len(cluster_counts)}")
    print(f"   - Largest cluster size: {cluster_counts.iloc[0]} points")
    print(f"   - Noise points: {sum(work_clusters == -1)} points")
    
else:
    print("⚠️ No significant location clusters found during work hours")
    work_location_inferred = None

In [ ]:
# 2. Identify Home Location (Most frequent location during nighttime hours 10pm-6am)

# Filter GPS data for nighttime hours (10pm - 6am)
night_hours_data = gps_df[
    (gps_df['hour'] >= 22) | (gps_df['hour'] <= 6)
].copy()

print(f"📊 Analyzing {len(night_hours_data)} GPS points during nighttime hours (10pm-6am)")

# Use DBSCAN clustering for nighttime locations
night_coords = night_hours_data[['latitude', 'longitude']].values

dbscan_night = DBSCAN(eps=eps_degrees, min_samples=min_samples)
night_clusters = dbscan_night.fit_predict(night_coords)

night_hours_data['cluster'] = night_clusters

# Find the most frequent cluster during nighttime
night_cluster_counts = pd.Series(night_clusters).value_counts()
night_cluster_counts = night_cluster_counts[night_cluster_counts.index != -1]  # Remove noise

if len(night_cluster_counts) > 0:
    most_frequent_night_cluster = night_cluster_counts.index[0]
    home_cluster_data = night_hours_data[night_hours_data['cluster'] == most_frequent_night_cluster]
    
    # Calculate centroid of the most frequent nighttime cluster
    home_location_inferred = [
        home_cluster_data['latitude'].mean(),
        home_cluster_data['longitude'].mean()
    ]
    
    print(f"🏠 Inferred HOME location: ({home_location_inferred[0]:.6f}, {home_location_inferred[1]:.6f})")
    print(f"📍 Based on {len(home_cluster_data)} GPS points in largest cluster")
    print(f"🕒 Time range: {home_cluster_data['timestamp'].min()} to {home_cluster_data['timestamp'].max()}")
    
    # Show cluster statistics
    print(f"\n📊 Night Hours Clustering Results:")
    print(f"   - Total clusters found: {len(night_cluster_counts)}")
    print(f"   - Largest cluster size: {night_cluster_counts.iloc[0]} points")
    print(f"   - Noise points: {sum(night_clusters == -1)} points")
    
else:
    print("⚠️ No significant location clusters found during nighttime hours")
    home_location_inferred = None

In [ ]:
# 3. Calculate distance between inferred work and home locations

if work_location_inferred and home_location_inferred:
    # Calculate geodesic distance between work and home
    home_work_distance = geodesic(home_location_inferred, work_location_inferred).kilometers
    
    print(f"🗺️ MOBILITY ANALYSIS RESULTS")
    print(f"=" * 50)
    print(f"🏠 Home Location: ({home_location_inferred[0]:.6f}, {home_location_inferred[1]:.6f})")
    print(f"🏢 Work Location: ({work_location_inferred[0]:.6f}, {work_location_inferred[1]:.6f})")
    print(f"📏 Home-Work Distance: {home_work_distance:.2f} km")
    
    # Classify commute distance for credit risk assessment
    if home_work_distance < 5:
        commute_risk = "Low"
        commute_desc = "Short commute - likely stable local employment"
    elif home_work_distance < 15:
        commute_risk = "Medium"
        commute_desc = "Moderate commute - normal urban pattern"
    elif home_work_distance < 30:
        commute_risk = "Medium-High"
        commute_desc = "Long commute - may indicate job instability or high transport costs"
    else:
        commute_risk = "High"
        commute_desc = "Very long commute - potential employment or financial stress"
    
    print(f"🚗 Commute Risk Level: {commute_risk}")
    print(f"💡 Interpretation: {commute_desc}")
    
    # Additional mobility metrics for credit assessment
    work_day_consistency = len(work_cluster_data) / len(work_hours_data) * 100
    home_night_consistency = len(home_cluster_data) / len(night_hours_data) * 100
    
    print(f"\n📊 STABILITY METRICS:")
    print(f"   - Work Location Consistency: {work_day_consistency:.1f}% of work hours")
    print(f"   - Home Location Consistency: {home_night_consistency:.1f}% of night hours")
    
    if work_day_consistency > 70 and home_night_consistency > 80:
        stability_score = "High"
        stability_desc = "Regular patterns suggest employment and residential stability"
    elif work_day_consistency > 50 and home_night_consistency > 60:
        stability_score = "Medium"
        stability_desc = "Moderate regularity in daily patterns"
    else:
        stability_score = "Low"
        stability_desc = "Irregular patterns may indicate instability"
    
    print(f"📈 Stability Score: {stability_score}")
    print(f"💡 Assessment: {stability_desc}")
    
else:
    print("❌ Could not calculate distance - insufficient location data")

In [ ]:
# 4. Visualize GPS trajectory patterns and identified locations

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🗺️ GPS Mobility Pattern Analysis', fontsize=16, fontweight='bold')

# Plot 1: All GPS points colored by time of day
ax1 = axes[0, 0]
scatter = ax1.scatter(gps_df['longitude'], gps_df['latitude'], 
                     c=gps_df['hour'], cmap='viridis', alpha=0.6, s=20)
plt.colorbar(scatter, ax=ax1, label='Hour of Day')
ax1.set_title('📍 All GPS Trajectories by Time of Day')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

# Plot 2: Work hours clustering
ax2 = axes[0, 1]
if work_location_inferred:
    # Color by cluster
    colors = ['red' if c == -1 else plt.cm.Set1(c) for c in work_hours_data['cluster']]
    ax2.scatter(work_hours_data['longitude'], work_hours_data['latitude'], 
               c=colors, alpha=0.7, s=30)
    ax2.scatter(work_location_inferred[1], work_location_inferred[0], 
               color='red', s=200, marker='*', label='Inferred Work Location')
    ax2.legend()
ax2.set_title('🏢 Work Hours Clustering (9am-5pm)')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')

# Plot 3: Night hours clustering
ax3 = axes[1, 0]
if home_location_inferred:
    colors = ['red' if c == -1 else plt.cm.Set1(c) for c in night_hours_data['cluster']]
    ax3.scatter(night_hours_data['longitude'], night_hours_data['latitude'], 
               c=colors, alpha=0.7, s=30)
    ax3.scatter(home_location_inferred[1], home_location_inferred[0], 
               color='blue', s=200, marker='*', label='Inferred Home Location')
    ax3.legend()
ax3.set_title('🏠 Night Hours Clustering (10pm-6am)')
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')

# Plot 4: Home-Work relationship
ax4 = axes[1, 1]
if work_location_inferred and home_location_inferred:
    # Plot all points in light gray
    ax4.scatter(gps_df['longitude'], gps_df['latitude'], 
               color='lightgray', alpha=0.3, s=10, label='All GPS Points')
    
    # Highlight home and work locations
    ax4.scatter(home_location_inferred[1], home_location_inferred[0], 
               color='blue', s=200, marker='H', label=f'Home Location')
    ax4.scatter(work_location_inferred[1], work_location_inferred[0], 
               color='red', s=200, marker='W', label=f'Work Location')
    
    # Draw line between home and work
    ax4.plot([home_location_inferred[1], work_location_inferred[1]], 
            [home_location_inferred[0], work_location_inferred[0]], 
            'g--', linewidth=2, alpha=0.7, 
            label=f'Distance: {home_work_distance:.2f} km')
    
    ax4.legend()
ax4.set_title('🚗 Home-Work Commute Pattern')
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')

plt.tight_layout()
plt.show()

# Create a summary mobility features dataframe
if work_location_inferred and home_location_inferred:
    mobility_features = pd.DataFrame({
        'user_id': ['user_001'],
        'home_lat': [home_location_inferred[0]],
        'home_lon': [home_location_inferred[1]],
        'work_lat': [work_location_inferred[0]],
        'work_lon': [work_location_inferred[1]],
        'commute_distance_km': [home_work_distance],
        'work_consistency_pct': [work_day_consistency],
        'home_consistency_pct': [home_night_consistency],
        'commute_risk_level': [commute_risk],
        'stability_score': [stability_score]
    })
    
    print("\n📊 MOBILITY FEATURES SUMMARY:")
    print(mobility_features.round(4))

In [ ]:
# 5. Advanced Mobility Features for Credit Risk Assessment

def extract_advanced_mobility_features(gps_data):
    """
    Extract comprehensive mobility features relevant for credit risk assessment
    """
    features = {}
    
    # Basic location statistics
    features['total_locations'] = len(gps_data)
    features['unique_days'] = gps_data['date'].nunique()
    features['avg_points_per_day'] = features['total_locations'] / features['unique_days']
    
    # Geographic spread
    lat_range = gps_data['latitude'].max() - gps_data['latitude'].min()
    lon_range = gps_data['longitude'].max() - gps_data['longitude'].min()
    features['location_spread_lat'] = lat_range
    features['location_spread_lon'] = lon_range
    features['location_spread_total'] = lat_range + lon_range
    
    # Time-based patterns
    hourly_activity = gps_data.groupby('hour').size()
    features['peak_activity_hour'] = hourly_activity.idxmax()
    features['activity_variance'] = hourly_activity.var()
    
    # Weekend vs weekday patterns
    gps_data['weekday'] = pd.to_datetime(gps_data['timestamp']).dt.weekday
    weekday_data = gps_data[gps_data['weekday'] < 5]  # Mon-Fri
    weekend_data = gps_data[gps_data['weekday'] >= 5]  # Sat-Sun
    
    if len(weekday_data) > 0 and len(weekend_data) > 0:
        features['weekday_mobility'] = len(weekday_data) / 5  # avg per weekday
        features['weekend_mobility'] = len(weekend_data) / 2  # avg per weekend day
        features['weekend_weekday_ratio'] = features['weekend_mobility'] / features['weekday_mobility']
    
    # Regular pattern detection
    daily_patterns = gps_data.groupby(['date', 'hour']).size().reset_index(name='count')
    pattern_consistency = daily_patterns.groupby('hour')['count'].std().mean()
    features['pattern_consistency'] = pattern_consistency
    
    # Mobility radius (distance from centroid)
    centroid_lat = gps_data['latitude'].mean()
    centroid_lon = gps_data['longitude'].mean()
    distances = gps_data.apply(
        lambda row: geodesic((centroid_lat, centroid_lon), 
                           (row['latitude'], row['longitude'])).kilometers, 
        axis=1
    )
    features['avg_distance_from_center'] = distances.mean()
    features['max_distance_from_center'] = distances.max()
    features['mobility_radius_95th'] = distances.quantile(0.95)
    
    return features

# Extract advanced features
advanced_features = extract_advanced_mobility_features(gps_df)

print("🔍 ADVANCED MOBILITY FEATURES:")
print("=" * 50)
for feature, value in advanced_features.items():
    if isinstance(value, float):
        print(f"{feature:.<30} {value:.3f}")
    else:
        print(f"{feature:.<30} {value}")

# Create risk assessment based on mobility patterns
def assess_mobility_risk(features, commute_distance=None):
    """
    Assess credit risk based on mobility patterns
    """
    risk_factors = []
    risk_score = 0
    
    # Factor 1: Commute distance
    if commute_distance:
        if commute_distance > 30:
            risk_factors.append("Very long commute (>30km)")
            risk_score += 2
        elif commute_distance > 15:
            risk_factors.append("Long commute (15-30km)")
            risk_score += 1
    
    # Factor 2: Location spread (higher spread = more unstable)
    if features['location_spread_total'] > 0.05:
        risk_factors.append("High geographic dispersion")
        risk_score += 1
    
    # Factor 3: Activity consistency
    if features['pattern_consistency'] > 2:
        risk_factors.append("Irregular daily patterns")
        risk_score += 1
    
    # Factor 4: Mobility radius
    if features['mobility_radius_95th'] > 20:
        risk_factors.append("Very wide mobility range")
        risk_score += 1
    
    # Factor 5: Weekend/weekday ratio
    if 'weekend_weekday_ratio' in features:
        if features['weekend_weekday_ratio'] > 2:
            risk_factors.append("Unusual weekend activity pattern")
            risk_score += 1
    
    # Determine overall risk level
    if risk_score <= 1:
        risk_level = "Low"
        risk_description = "Stable mobility patterns indicate residential and employment stability"
    elif risk_score <= 3:
        risk_level = "Medium"
        risk_description = "Some irregular patterns but generally stable"
    else:
        risk_level = "High"
        risk_description = "Multiple indicators of instability in mobility patterns"
    
    return {
        'risk_score': risk_score,
        'risk_level': risk_level,
        'risk_factors': risk_factors,
        'description': risk_description
    }

# Perform risk assessment
mobility_risk = assess_mobility_risk(advanced_features, 
                                   home_work_distance if 'home_work_distance' in locals() else None)

print(f"\n🎯 MOBILITY RISK ASSESSMENT:")
print(f"=" * 50)
print(f"Risk Level: {mobility_risk['risk_level']}")
print(f"Risk Score: {mobility_risk['risk_score']}/6")
print(f"Description: {mobility_risk['description']}")

if mobility_risk['risk_factors']:
    print(f"\nRisk Factors Identified:")
    for factor in mobility_risk['risk_factors']:
        print(f"  ⚠️ {factor}")
else:
    print(f"\n✅ No significant risk factors identified")

print(f"\n💡 Credit Risk Implications:")
print(f"   - Regular home/work patterns suggest employment stability")
print(f"   - Reasonable commute distance indicates sustainable lifestyle")
print(f"   - Consistent daily patterns show predictable behavior")
print(f"   - Limited mobility radius suggests local community ties")

# 📱 Device Features - Mobile Usage Analysis

Device usage patterns can provide valuable insights for credit risk assessment:
- **Device Quality**: Higher-end devices may indicate better financial status
- **Usage Patterns**: App usage and screen time reveal lifestyle and spending habits
- **Data Consumption**: Heavy data usage might indicate higher income or tech-savvy users
- **Battery Behavior**: Usage intensity patterns show daily routine consistency

We'll prototype features using mobile device usage data to extract:
1. Operating System and Device Model encoding
2. Data usage efficiency metrics 
3. Battery consumption patterns
4. App usage intensity features

In [ ]:
# Since we don't have access to the actual Mobile Device Usage dataset, we'll simulate realistic device data
# This mimics the structure of mobile device usage datasets commonly found on Kaggle

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

# Generate realistic mobile device usage data
n_users = 1000

# Device specifications
operating_systems = ['iOS', 'Android', 'Other']
os_weights = [0.4, 0.55, 0.05]

device_models = [
    'iPhone 14', 'iPhone 13', 'iPhone 12', 'iPhone SE',
    'Samsung Galaxy S23', 'Samsung Galaxy A54', 'Samsung Galaxy A34',
    'Google Pixel 7', 'Google Pixel 6a', 'OnePlus 11',
    'Xiaomi 13', 'Huawei P50', 'Other Android', 'Other Device'
]

# Generate device data
device_data = []

for i in range(n_users):
    # Select OS first, then compatible device
    os = np.random.choice(operating_systems, p=os_weights)
    
    if os == 'iOS':
        device = np.random.choice(['iPhone 14', 'iPhone 13', 'iPhone 12', 'iPhone SE'], 
                                p=[0.3, 0.35, 0.25, 0.1])
    elif os == 'Android':
        device = np.random.choice([
            'Samsung Galaxy S23', 'Samsung Galaxy A54', 'Samsung Galaxy A34',
            'Google Pixel 7', 'Google Pixel 6a', 'OnePlus 11',
            'Xiaomi 13', 'Huawei P50', 'Other Android'
        ], p=[0.2, 0.15, 0.15, 0.1, 0.1, 0.08, 0.08, 0.05, 0.09])
    else:
        device = 'Other Device'
    
    # Generate usage patterns with some correlation to device type
    device_tier = 'premium' if device in ['iPhone 14', 'iPhone 13', 'Samsung Galaxy S23', 'Google Pixel 7', 'OnePlus 11'] else 'standard'
    
    # App usage patterns
    if device_tier == 'premium':
        screen_time = np.random.normal(6.5, 2.0)  # Premium users tend to use phones more
        num_apps = np.random.poisson(45) + 20
        data_usage_mb = np.random.normal(8500, 2500)
    else:
        screen_time = np.random.normal(4.8, 1.8)
        num_apps = np.random.poisson(30) + 15
        data_usage_mb = np.random.normal(5500, 2000)
    
    # Ensure positive values
    screen_time = max(1.0, screen_time)
    num_apps = max(10, num_apps)
    data_usage_mb = max(500, data_usage_mb)
    
    # Battery and usage patterns
    battery_drain = np.random.normal(25, 8)  # % per day
    battery_drain = np.clip(battery_drain, 10, 50)
    
    app_usage_time = screen_time * np.random.uniform(0.7, 0.9)  # Apps don't account for all screen time
    
    # Age affects usage patterns
    user_age = np.random.randint(18, 65)
    if user_age < 30:
        # Younger users are more active
        screen_time *= np.random.uniform(1.1, 1.4)
        data_usage_mb *= np.random.uniform(1.2, 1.5)
    elif user_age > 50:
        # Older users are less active
        screen_time *= np.random.uniform(0.6, 0.9)
        data_usage_mb *= np.random.uniform(0.7, 0.9)
    
    device_data.append({
        'User_ID': f'USER_{i+1:04d}',
        'Device_Model': device,
        'Operating_System': os,
        'App_Usage_Time_hours': round(app_usage_time, 2),
        'Screen_On_Time_hours': round(screen_time, 2),
        'Battery_Drain_percent': round(battery_drain, 1),
        'Number_of_Apps_Installed': int(num_apps),
        'Data_Usage_MB': round(data_usage_mb, 1),
        'Age': user_age,
        'Device_Tier': device_tier  # For reference, normally not available
    })

# Create DataFrame
device_df = pd.DataFrame(device_data)

print(f"📊 Generated mobile device usage data for {len(device_df)} users")
print(f"📱 Operating Systems: {device_df['Operating_System'].value_counts().to_dict()}")
print(f"📱 Device Models: {device_df['Device_Model'].nunique()} unique models")
print(f"📊 Average screen time: {device_df['Screen_On_Time_hours'].mean():.2f} hours/day")
print(f"📊 Average data usage: {device_df['Data_Usage_MB'].mean():.0f} MB/month")

device_df.head(10)

In [ ]:
# 1. One-hot encode Operating System and Device Model columns

# Create a copy for encoding
device_encoded = device_df.copy()

print("🔄 One-Hot Encoding Device Features")
print("=" * 50)

# Operating System One-Hot Encoding
print("📱 Operating System encoding...")
os_encoded = pd.get_dummies(device_encoded['Operating_System'], prefix='OS')
print(f"   Created {len(os_encoded.columns)} OS features: {list(os_encoded.columns)}")

# Device Model One-Hot Encoding  
print("📱 Device Model encoding...")
device_model_encoded = pd.get_dummies(device_encoded['Device_Model'], prefix='Device')
print(f"   Created {len(device_model_encoded.columns)} device features")
print(f"   Device features: {list(device_model_encoded.columns)}")

# Combine original features with encoded features
device_encoded = pd.concat([
    device_encoded.drop(['Operating_System', 'Device_Model'], axis=1),
    os_encoded,
    device_model_encoded
], axis=1)

print(f"\n📊 Enhanced dataset shape: {device_encoded.shape}")
print(f"📊 Original features: {device_df.shape[1]}")
print(f"📊 New features added: {device_encoded.shape[1] - device_df.shape[1]}")

# Show the distribution of encoded features
print(f"\n📈 Operating System Distribution:")
for col in os_encoded.columns:
    count = os_encoded[col].sum()
    pct = count / len(device_encoded) * 100
    print(f"   {col}: {count} users ({pct:.1f}%)")

print(f"\n📈 Top Device Models:")
device_counts = device_df['Device_Model'].value_counts().head(8)
for device, count in device_counts.items():
    pct = count / len(device_df) * 100
    print(f"   {device}: {count} users ({pct:.1f}%)")

device_encoded.head()

In [ ]:
# 2. Create interaction features for credit risk assessment

print("⚙️ Creating Device Interaction Features")
print("=" * 50)

# Create interaction features
device_features = device_encoded.copy()

# Key interaction features as specified
print("📊 Core Interaction Features:")

# 1. Data usage per app
device_features['data_usage_per_app'] = (
    device_features['Data_Usage_MB'] / device_features['Number_of_Apps_Installed']
)
print(f"   ✅ data_usage_per_app: Data Usage / Number of Apps")
print(f"      Range: {device_features['data_usage_per_app'].min():.1f} - {device_features['data_usage_per_app'].max():.1f} MB/app")

# 2. Battery intensiveness  
device_features['battery_intensiveness'] = (
    device_features['Battery_Drain_percent'] / device_features['App_Usage_Time_hours']
)
print(f"   ✅ battery_intensiveness: Battery Drain / App Usage Time")
print(f"      Range: {device_features['battery_intensiveness'].min():.2f} - {device_features['battery_intensiveness'].max():.2f} %/hour")

# Additional valuable interaction features for credit assessment
print(f"\n📊 Additional Credit-Relevant Features:")

# 3. Screen efficiency (screen time vs app usage time)
device_features['screen_efficiency'] = (
    device_features['App_Usage_Time_hours'] / device_features['Screen_On_Time_hours']
)
print(f"   ✅ screen_efficiency: App Usage / Screen Time ratio")

# 4. App density (how many apps per hour of usage)
device_features['app_density'] = (
    device_features['Number_of_Apps_Installed'] / device_features['App_Usage_Time_hours']
)
print(f"   ✅ app_density: Apps per hour of usage")

# 5. Data intensity (data usage per hour)
device_features['data_intensity'] = (
    device_features['Data_Usage_MB'] / device_features['Screen_On_Time_hours']
)
print(f"   ✅ data_intensity: Data usage per screen hour")

# 6. Usage consistency (ratio of app usage to screen time)
device_features['usage_consistency'] = (
    device_features['App_Usage_Time_hours'] / device_features['Screen_On_Time_hours']
)
print(f"   ✅ usage_consistency: How focused is device usage")

# 7. Power efficiency score
device_features['power_efficiency'] = (
    device_features['Screen_On_Time_hours'] / device_features['Battery_Drain_percent'] * 100
)
print(f"   ✅ power_efficiency: Screen hours per battery %")

# Handle any infinite or NaN values
device_features = device_features.replace([np.inf, -np.inf], np.nan)
numeric_cols = ['data_usage_per_app', 'battery_intensiveness', 'screen_efficiency', 
                'app_density', 'data_intensity', 'usage_consistency', 'power_efficiency']

for col in numeric_cols:
    median_val = device_features[col].median()
    device_features[col] = device_features[col].fillna(median_val)

print(f"\n📈 Feature Statistics:")
for col in numeric_cols:
    values = device_features[col]
    print(f"   {col:.<25} {values.mean():.3f} ± {values.std():.3f}")

print(f"\n📊 Enhanced dataset now has {device_features.shape[1]} features")
device_features[['User_ID'] + numeric_cols].head(10)

In [ ]:
# 3. Visualize device features and analyze patterns for credit risk

fig, axes = plt.subplots(3, 2, figsize=(15, 18))
fig.suptitle('📱 Device Usage Pattern Analysis for Credit Risk', fontsize=16, fontweight='bold')

# Plot 1: Operating System vs Data Usage Per App
ax1 = axes[0, 0]
os_data_usage = []
os_labels = []
for os in device_features.columns:
    if os.startswith('OS_'):
        os_name = os.replace('OS_', '')
        os_users = device_features[device_features[os] == 1]
        if len(os_users) > 0:
            os_data_usage.append(os_users['data_usage_per_app'].values)
            os_labels.append(os_name)

ax1.boxplot(os_data_usage, labels=os_labels)
ax1.set_title('📊 Data Usage per App by Operating System')
ax1.set_ylabel('Data Usage per App (MB)')
ax1.tick_params(axis='x', rotation=45)

# Plot 2: Battery Intensiveness Distribution
ax2 = axes[0, 1]
ax2.hist(device_features['battery_intensiveness'], bins=30, alpha=0.7, color='orange')
ax2.set_title('🔋 Battery Intensiveness Distribution')
ax2.set_xlabel('Battery Drain per Usage Hour (%/hour)')
ax2.set_ylabel('Number of Users')
ax2.axvline(device_features['battery_intensiveness'].median(), color='red', linestyle='--', 
            label=f'Median: {device_features["battery_intensiveness"].median():.2f}')
ax2.legend()

# Plot 3: Device Tier vs Usage Patterns
ax3 = axes[1, 0]
premium_devices = ['iPhone 14', 'iPhone 13', 'Samsung Galaxy S23', 'Google Pixel 7', 'OnePlus 11']
device_features['is_premium'] = device_features['Device_Tier'] == 'premium'

premium_usage = device_features[device_features['is_premium']]['Screen_On_Time_hours']
standard_usage = device_features[~device_features['is_premium']]['Screen_On_Time_hours']

ax3.boxplot([premium_usage, standard_usage], labels=['Premium Devices', 'Standard Devices'])
ax3.set_title('📱 Screen Time by Device Tier')
ax3.set_ylabel('Screen Time (hours/day)')

# Plot 4: Age vs Data Intensity
ax4 = axes[1, 1]
scatter = ax4.scatter(device_features['Age'], device_features['data_intensity'], 
                     c=device_features['is_premium'], cmap='viridis', alpha=0.6)
ax4.set_title('📊 Age vs Data Intensity (Colored by Device Tier)')
ax4.set_xlabel('User Age')
ax4.set_ylabel('Data Intensity (MB/hour)')
plt.colorbar(scatter, ax=ax4, label='Premium Device (1=Yes, 0=No)')

# Plot 5: Feature Correlation Heatmap
ax5 = axes[2, 0]
correlation_features = ['data_usage_per_app', 'battery_intensiveness', 'screen_efficiency',
                       'app_density', 'data_intensity', 'usage_consistency', 'power_efficiency']
corr_matrix = device_features[correlation_features].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=ax5, fmt='.2f')
ax5.set_title('🔄 Device Feature Correlations')

# Plot 6: Credit Risk Indicators
ax6 = axes[2, 1]
# Create a simple risk score based on device patterns
device_features['risk_score'] = (
    (device_features['data_usage_per_app'] > device_features['data_usage_per_app'].quantile(0.8)).astype(int) +
    (device_features['battery_intensiveness'] > device_features['battery_intensiveness'].quantile(0.8)).astype(int) +
    (device_features['Screen_On_Time_hours'] > device_features['Screen_On_Time_hours'].quantile(0.8)).astype(int) +
    (device_features['is_premium']).astype(int) * -1  # Premium devices reduce risk
)

risk_counts = device_features['risk_score'].value_counts().sort_index()
ax6.bar(risk_counts.index, risk_counts.values, color=['green', 'yellow', 'orange', 'red'][:len(risk_counts)])
ax6.set_title('📈 Device-Based Risk Score Distribution')
ax6.set_xlabel('Risk Score')
ax6.set_ylabel('Number of Users')

plt.tight_layout()
plt.show()

# Summary statistics by device characteristics
print("\n📊 DEVICE FEATURES ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n💰 Premium vs Standard Device Analysis:")
premium_stats = device_features[device_features['is_premium']].agg({
    'data_usage_per_app': 'mean',
    'battery_intensiveness': 'mean', 
    'Screen_On_Time_hours': 'mean',
    'Data_Usage_MB': 'mean'
})
standard_stats = device_features[~device_features['is_premium']].agg({
    'data_usage_per_app': 'mean',
    'battery_intensiveness': 'mean',
    'Screen_On_Time_hours': 'mean', 
    'Data_Usage_MB': 'mean'
})

for feature in premium_stats.index:
    print(f"   {feature:.<25} Premium: {premium_stats[feature]:.2f} | Standard: {standard_stats[feature]:.2f}")

print(f"\n📱 Operating System Patterns:")
for os_col in [col for col in device_features.columns if col.startswith('OS_')]:
    os_name = os_col.replace('OS_', '')
    os_users = device_features[device_features[os_col] == 1]
    if len(os_users) > 0:
        avg_data_per_app = os_users['data_usage_per_app'].mean()
        avg_battery_int = os_users['battery_intensiveness'].mean()
        print(f"   {os_name:.<15} Data/App: {avg_data_per_app:.1f} MB | Battery Int: {avg_battery_int:.2f} %/hr")

In [ ]:
# 4. Credit Risk Assessment using Device Features

def assess_credit_risk_from_device(row):
    """
    Assess credit risk based on device usage patterns.
    Returns risk level and contributing factors.
    """
    risk_factors = []
    risk_score = 0
    
    # Device quality indicator
    if row['is_premium']:
        risk_score -= 1  # Premium devices suggest better financial status
    else:
        risk_factors.append("Standard device tier")
    
    # Data usage patterns
    if row['data_usage_per_app'] > row['data_usage_per_app'] * 1.5:  # High data usage per app
        risk_factors.append("High data usage per app")
        risk_score += 1
    
    # Battery usage patterns (very high might indicate financial stress - old device)
    if row['battery_intensiveness'] > device_features['battery_intensiveness'].quantile(0.8):
        risk_factors.append("High battery intensiveness")
        risk_score += 1
    
    # Excessive screen time might indicate unemployment or underemployment
    if row['Screen_On_Time_hours'] > device_features['Screen_On_Time_hours'].quantile(0.9):
        risk_factors.append("Excessive screen time")
        risk_score += 1
    
    # Very low screen time might indicate device issues or financial constraints
    if row['Screen_On_Time_hours'] < device_features['Screen_On_Time_hours'].quantile(0.1):
        risk_factors.append("Very low device usage")
        risk_score += 1
    
    # App efficiency (many apps but low usage might indicate tech-savvy user)
    if row['app_density'] > device_features['app_density'].quantile(0.8):
        risk_score -= 0.5  # Slightly positive indicator
    
    # Data intensity (high data usage suggests engagement and potentially higher income)
    if row['data_intensity'] > device_features['data_intensity'].quantile(0.7):
        risk_score -= 0.5  # Slightly positive indicator
    
    # Age-based adjustments
    if row['Age'] < 25:
        # Young users with premium devices and high usage are typically lower risk
        if row['is_premium'] and row['Screen_On_Time_hours'] > 4:
            risk_score -= 0.5
    elif row['Age'] > 55:
        # Older users with consistent moderate usage are typically stable
        if 2 <= row['Screen_On_Time_hours'] <= 5:
            risk_score -= 0.5
    
    # Determine risk level
    if risk_score <= 0:
        risk_level = "LOW"
    elif risk_score <= 2:
        risk_level = "MEDIUM"
    else:
        risk_level = "HIGH"
    
    return risk_level, risk_score, risk_factors

# Apply risk assessment to all users
print("🎯 DEVICE-BASED CREDIT RISK ASSESSMENT")
print("=" * 60)

risk_results = []
for idx, row in device_features.iterrows():
    risk_level, risk_score, risk_factors = assess_credit_risk_from_device(row)
    risk_results.append({
        'User_ID': row['User_ID'],
        'Risk_Level': risk_level,
        'Risk_Score': risk_score,
        'Risk_Factors': risk_factors
    })

risk_df = pd.DataFrame(risk_results)

# Summary statistics
risk_distribution = risk_df['Risk_Level'].value_counts()
print(f"📊 Risk Distribution:")
for level, count in risk_distribution.items():
    pct = count / len(risk_df) * 100
    print(f"   {level:.<10} {count:>4} users ({pct:>5.1f}%)")

# Show examples by risk level
print(f"\n👥 Sample Risk Assessments:")
for risk_level in ['LOW', 'MEDIUM', 'HIGH']:
    sample = risk_df[risk_df['Risk_Level'] == risk_level].head(3)
    print(f"\n🎯 {risk_level} RISK Examples:")
    
    for idx, row in sample.iterrows():
        user_data = device_features[device_features['User_ID'] == row['User_ID']].iloc[0]
        print(f"   👤 {row['User_ID']}:")
        print(f"      📱 Device: {user_data['Device_Tier']} ({device_df[device_df['User_ID'] == row['User_ID']]['Device_Model'].iloc[0]})")
        print(f"      📊 Screen Time: {user_data['Screen_On_Time_hours']:.1f} hrs")
        print(f"      📶 Data/App: {user_data['data_usage_per_app']:.1f} MB")
        print(f"      🔋 Battery Int.: {user_data['battery_intensiveness']:.2f} %/hr")
        print(f"      👤 Age: {user_data['Age']}")
        print(f"      ⚠️  Risk Factors: {', '.join(row['Risk_Factors']) if row['Risk_Factors'] else 'None'}")

# Device feature importance for credit decisions
print(f"\n📈 DEVICE FEATURES FOR CREDIT ASSESSMENT:")
print("-" * 60)

feature_importance = {
    'data_usage_per_app': 'Data efficiency indicates financial awareness',
    'battery_intensiveness': 'High values may suggest old/damaged device',
    'screen_efficiency': 'Focused usage suggests purpose-driven behavior',
    'app_density': 'Tech-savvy users often have better financial literacy',
    'data_intensity': 'High data usage correlates with engagement and income',
    'usage_consistency': 'Regular patterns indicate lifestyle stability',
    'power_efficiency': 'Device maintenance suggests financial responsibility'
}

for feature, description in feature_importance.items():
    avg_val = device_features[feature].mean()
    print(f"   📊 {feature:.<25} {description}")
    print(f"      {'':.<25} Average: {avg_val:.3f}")

print(f"\n💡 Credit Risk Insights from Device Data:")
print(f"   ✅ Premium device users show {(risk_df[device_features['is_premium']]['Risk_Level'] == 'LOW').mean()*100:.1f}% low risk rate")
print(f"   📱 iOS users have {device_features[device_features['OS_iOS'] == 1]['data_usage_per_app'].mean():.1f} MB/app average")
print(f"   🤖 Android users have {device_features[device_features['OS_Android'] == 1]['data_usage_per_app'].mean():.1f} MB/app average")
print(f"   ⚡ High battery intensiveness (>{device_features['battery_intensiveness'].quantile(0.8):.2f}) affects {(device_features['battery_intensiveness'] > device_features['battery_intensiveness'].quantile(0.8)).sum()} users")

# Create final device feature summary for credit model
device_summary = device_features.groupby('User_ID').agg({
    'data_usage_per_app': 'first',
    'battery_intensiveness': 'first',
    'screen_efficiency': 'first',
    'app_density': 'first',
    'data_intensity': 'first',
    'usage_consistency': 'first',
    'power_efficiency': 'first',
    'is_premium': 'first'
}).round(3)

# Add OS and device encoding
os_cols = [col for col in device_features.columns if col.startswith('OS_')]
device_cols = [col for col in device_features.columns if col.startswith('Device_')]

for col in os_cols + device_cols:
    device_summary[col] = device_features.groupby('User_ID')[col].first()

print(f"\n📋 Device Feature Summary for Credit Model:")
print(f"   📊 Features per user: {device_summary.shape[1]}")
print(f"   👥 Users analyzed: {device_summary.shape[0]}")

device_summary.head()